In [9]:
import httpx
import time
import pandas as pd

In [27]:
datacenters_keywords = ["datacenter", "data center", "datacentre", "data centre"]

environmental_keywords = ["environment", "pollution", "emissions", "diesel", "fumes", "light pollution", "water", "drought", "water consumption",  "waste", "carbon", "climate"]
infrastructure_utilities_keywords = ["utilities", "utility", "electric", "power", "grid", "blackout", "substation", "reliable", "reliability", "capacity", "infrastructure", "generator", "water"]
housing_keywords = ["housing", "house", "home", "rent", "property", "real estate", "house price", "housing price", "housing cost", "land cost", "land price", "gentrification"]
economic_keywords = ["job", "work", "revenue", "tax", "money", "rich", "salary", "economy", "investment", "business"]
life_quality_keywords = ["resident", "fumes", "noise", "loud", "vibration", "light pollution", "sound", "smell", "traffic"]
aesthetics_keywords = ["landscape", "visual impact",  "aesthetics", "looks", "architecture", "design", "facade", "industrial", "windows", "dark"]
government_keywords = ["zoning", "planning", "policy", "permit", "city council", "bill", "mayor", "governor", "government", "legislat"]

HEADERS = {
    "User-Agent": "research-script/0.1 (academic, non-commercial)"
}

In [32]:
print("Retrieving post from id:\n")

with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
        pid = "1qborpq"
        post_url = f"https://www.reddit.com/comments/{pid}.json"                
        r = client.get(post_url)            
        post_json = r.json()
        text = post_json[0]["data"]["children"][0]["data"]["selftext"] + post_json[0]["data"]["children"][0]["data"]["title"]

if any(kw in text.lower() for kw in datacenters_keywords):
   print("Text contains the word datacenter")

Retrieving post from id:

Text contains the word datacenter


In [33]:
print("Original method of classifying themes, word bank provided by me:\n")

if any(kw in text for kw in environmental_keywords):
    print("Environmental")
    matched = True
if any(kw in text for kw in infrastructure_utilities_keywords):
    print("Infrastructure")
    matched = True
if any(kw in text for kw in housing_keywords):
    print("Housing")
    matched = True
if any(kw in text for kw in economic_keywords):
    print("Economic")
    matched = True
if any(kw in text for kw in life_quality_keywords):
    print("Life quality")
    matched = True
if any(kw in text for kw in aesthetics_keywords):
    print("Aesthetics")
    matched = True
if any(kw in text for kw in government_keywords):
    print("Governmental")
    matched = True
if matched == False:
    print("None")

Original method of classifying themes, word bank provided by me:

Infrastructure
Governmental


In [38]:
print("Using a model to classify themes:\n")

theme_classifier = pipeline(
    "zero-shot-classification",
    model = "knowledgator/comprehend_it-base"
)
labels = ["environment", "infrastructure", "housing", "economy", "quality of life", "visual impact", "government"]

theme_scores = theme_classifier(text, labels)["scores"]
print(theme_scores)
matched = False
average_metric = (sum(theme_scores)/7)-0.1*(sum(theme_scores)/7)
        
if theme_scores[0]>average_metric and theme_scores[0]>0.15:
    print("Environmental")
    matched = True
if theme_scores[1]>average_metric and theme_scores[1]>0.15:
    print("Infrastructure")
    matched = True
if theme_scores[2]>average_metric and theme_scores[2]>0.15:
    print("Housing")
    matched = True
if theme_scores[3]>average_metric and theme_scores[3]>0.15:
    print("Economic")
    matched = True
if theme_scores[4]>average_metric and theme_scores[4]>0.15:
    print("Life quality")
    matched = True
if theme_scores[5]>average_metric and theme_scores[5]>0.15:
    print("Aesthetics")
    matched = True
if theme_scores[6]>average_metric and theme_scores[6]>0.15:
    print("Governmental")
    matched = True
if not matched:
    print("No theme")

Using a model to classify themes:



Device set to use mps:0


[0.33794137835502625, 0.2230110466480255, 0.18559765815734863, 0.1310614049434662, 0.08292143791913986, 0.029485220089554787, 0.009981817565858364]
Environmental
Infrastructure
Housing


So far:

    For post id 1qborpq:
        My bag-of-words method returned the themes of infrastructure and government
        facebook/bart-large-mnli returned the themes of environment, infrastructure, and housing
        typeform/distilbert-base-uncased-mnli returned the themes of environmental and infrastructure
        knowledgator/comprehend_it-base returned the themes of environmental, infrastructure, and housing
        joeddav/bart-large-mnli-yahoo-answers returned no theme with a score of over 0.5

    For post id 1bdp55a (very governmental):
        My bag-of-words method returned the themes of environment, housing, life quality, aesthetics, and government
        facebook/bart-large-mnli returned the themes of environment and infrastructure
        joeddav/bart-large-mnli-yahoo-answers returned no theme with a score of over 0.5

In [13]:
print("Sentiment analysis:\n")

import re
from transformers import pipeline
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True,      # <--- important
    padding=True,         # <--- for batching
    max_length=512   
)

results = classifier(text, truncation=True, padding=True, max_length=512, batch_size=32)
print(results)

Sentiment analysis:



Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.7456856369972229}]
